### aim: check that all data 

### date: 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 13})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 13}
plt.rcParams['text.usetex'] = True
plt.rc('font', **font)

import arrow

## JRA

In [ ]:
tdir = '//gpfs/data/greenocean2/software/products/windsFromComponents/JRA/'

tf = 'jra3q.anl_surf.0_2_3.vgrd10m-hgt-an-gauss.1981030100_1981033118.nc'

# w =xr.open_dataset(f'{tdir}{tf}')
# # w

mos = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

def make_yearlists(yr):
    
    vlist = []
    ulist = []
    
    for m in range(0,12):
        tstr = f'{yr}{mos[m]}'
        
        t = glob.glob(f'{tdir}/jra3q.*vgrd10m*.{tstr}*.nc')
        ts = t[0]
        vlist.append(ts)
        
        t = glob.glob(f'{tdir}/jra3q.*ugrd10m*.{tstr}*.nc')
        ts = t[0]
        ulist.append(ts)
        
    return ulist, vlist


for yr in range(1980,2024):
    
    print(yr)
    
    try:
        
        ulist, vlist = make_yearlists(yr)

        ub = xr.open_mfdataset(ulist)
        vb = xr.open_mfdataset(vlist)
        print('opened files')
        # vb = xr.open_mfdataset(vlist)
        ub = ub.rename({"ugrd10m-hgt-an-gauss": "u10m"})
        vb = vb.rename({"vgrd10m-hgt-an-gauss": "v10m"})

        ub = ub['u10m'].resample(time='1D').mean()
        vb = vb['v10m'].resample(time='1D').mean()

        wspd = xr.ufuncs.sqrt(ub**2 + vb**2)
        wspd.name = 'wspd10m'
        wb = xr.merge([ub, vb, wspd])

        wb.to_netcdf(f'/gpfs/data/greenocean2/software/products/windsFromComponents/JRA/daily/JRA_{yr}_daily.nc')
        print('saved files')
    
    except:
        
        print(f'problem in yr {yr}')


1980
opened files
saved files
1981
opened files
problem in yr 1981
1982
opened files
saved files
1983
opened files
saved files
1984
opened files
saved files
1985
opened files
problem in yr 1985
1986
problem in yr 1986
1987
problem in yr 1987
1988


In [ ]:
wb

In [ ]:
ub

In [ ]:
ugrd = w['ugrd10m-hgt-an-gauss'].resample(time='1D').mean()
vgrd = w['ugrd10m-hgt-an-gauss'].resample(time='1D').mean()

In [ ]:


wspd = xr.ufuncs.sqrt(uwi.uwind10m**2 + vwi.vwind10m**2)
wspd.name = 'wspd10m'
wb = xr.merge([uwi, vwi, wspd])
wb = wb.rename({"uwind10m": "u10m"})
wb = wb.rename({"vwind10m": "v10m"})

In [ ]:
def get_daily(yr):

    date_list = []
    
    # Loop through all months and days of the year
    for month in range(1, 13):  # Months from 1 to 12
        for day in range(1, 32):  # Days from 1 to 31
            try:
                # Create a date with year, month, and day
                date = arrow.get(yr, month, day)
                # Format the date as YYYYMMDD
                formatted_date = date.format('YYYYMMDD')
                # Append to the list
                date_list.append(formatted_date)
            except:
                #print(f'no {month} {day}')
                # Skip invalid dates (e.g., 1980-02-30)
                continue
    
    print(len(date_list))
    print(date_list[364])

    for i in range(0,len(date_list)):
        td = date_list[i]
        # print(td)
        tl = glob.glob(f'/gpfs/data/greenocean2/software/products/windsFromComponents/NASA-MERRA2/raw/downloads/*{td}*nc*')
        print(tl)
        
        tdat = (tl[0])
        w = xr.open_dataset(tdat)
        
        # Resample both variables from hourly to daily, taking the mean for simplicity
        UL_daily = w['U10M'].resample(time='1D').mean()
        VL_daily = w['V10M'].resample(time='1D').mean()
        twind = xr.ufuncs.sqrt(xr.ufuncs.square(UL_daily) + xr.ufuncs.square(VL_daily))
        twind.name = 'windspeed'

        # Create a new dataset with the resampled variables
        new_ds = xr.Dataset({
            'u10m': UL_daily,
            'v10m': VL_daily,
            'wspd10m': twind,
        
            
        })
        
        newdat = f'/gpfs/data/greenocean2/software/products/windsFromComponents/NASA-MERRA2/daily/MERRA2_{td}_daily.nc'
        # if i%20 == 0:
        #     print(newdat)
        new_ds.to_netcdf(newdat)        
        

## test daily

In [ ]:
def test_daily(yr):

    date_list = []
    
    # Loop through all months and days of the year
    for month in range(1, 13):  # Months from 1 to 12
        for day in range(1, 32):  # Days from 1 to 31
            try:
                # Create a date with year, month, and day
                date = arrow.get(yr, month, day)
                # Format the date as YYYYMMDD
                formatted_date = date.format('YYYYMMDD')
                # Append to the list
                date_list.append(formatted_date)
            except:
                #print(f'no {month} {day}')
                # Skip invalid dates (e.g., 1980-02-30)
                continue
    
    print(len(date_list))
    print(date_list[364])

    for i in range(0,len(date_list)):
        td = date_list[i]
        # print(td)
        tl = glob.glob(f'/gpfs/data/greenocean2/software/products/windsFromComponents/NASA-MERRA2/raw/downloads/*{td}*nc*')
        #print(tl)
        try:
            tdat = (tl[0])
            w = xr.open_dataset(tdat)
        except:
            print(td)
    
        

In [ ]:
ex = False
if ex:
    # for yr in range(1981,2024):
    #     print(yr)
    #     test_daily(yr)
    for yr in range(1984,2024):
        print(yr)
        get_daily(yr)

In [ ]:
# ex = True
# if ex:
#     for yr in range(1981,2024):
#         print(yr)
#         get_daily(yr)
#     # for yr in range(1995,2024):
#     #     print(yr)
#     #     get_daily(yr)

In [ ]:
def get_daily_list(yr):

    date_list = []
    
    # Loop through all months and days of the year
    for month in range(1, 13):  # Months from 1 to 12
        for day in range(1, 32):  # Days from 1 to 31
            try:
                # Create a date with year, month, and day
                date = arrow.get(yr, month, day)
                # Format the date as YYYYMMDD
                formatted_date = date.format('YYYYMMDD')
                # Append to the list
                date_list.append(formatted_date)
            except:
                #print(f'no {month} {day}')
                # Skip invalid dates (e.g., 1980-02-30)
                continue
    
    print(len(date_list))
    print(date_list[364])

    dailylist = []
    for i in range(0,len(date_list)):
        td = date_list[i]
        tl = glob.glob(f'/gpfs/data/greenocean2/software/products/windsFromComponents/NASA-MERRA2/daily/MERRA2_{td}_daily.nc')
        tdat = (tl[0])
        dailylist.append(tdat)

    return dailylist


In [ ]:
ex = False
if ex:
    for y in range(1997,2024):

        dl = get_daily_list(y)
        w = xr.open_mfdataset(dl)
        w.to_netcdf(f'/gpfs/data/greenocean2/software/products/windsFromComponents/NASA-MERRA2/daily/MERRA2_FY_{y}_daily.nc')

In [ ]:
td = '/gpfs/data/greenocean2/software/products/windsFromComponents/dailyStandard/MERRA/'


ex = True
if ex:
    for y in range(1991,2024):

        w = xr.open_mfdataset(f'{td}/MERRA_wind_daily_1x1_{y}.nc')
        wr = w.rename({"time": "time_counter"})
        wr.to_netcdf(f'{td}/fix/MERRA_wind_daily_1x1_{y}.nc')